In [1]:
import sys
# Before using tensorflow:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import tensorflow as tf
import tensorflow.keras.backend as KB
from matplotlib import pylab as plt
import tensorflow.keras
#from tensorflow.python.framework.ops import disable_eager_execution
#disable_eager_execution()

import pickle
import numpy as np
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")

gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")


Tensor Flow Version: 2.4.1
Keras Version: 2.4.0

Python 3.8.5 (default, May 27 2021, 13:30:53) 
[GCC 9.3.0]
GPU is available


## Create model

In [2]:
import tensorflow
from client.models.AMLmodel import AMLModel, construct_model

#model = AMLModel()
model = construct_model()

In [ ]:
model.summary()

## Load training data

In [3]:
import numpy as np
labels = np.load('dataset/processed/data_partitions/partition0/labels.npy', allow_pickle=True).item()

In [4]:
ids = [l for l in labels]

In [5]:
from client.data.datagenerator import DataGenerator

In [7]:
data_path = 'dataset/processed/data_partitions/partition0/data_singlets'
train_gen = DataGenerator(ids,labels, data_path, dim=(100,100), batch_size=32)

In [8]:
model.fit(train_gen,epochs=10)

Epoch 1/10
57/57 [==============================] - 16s 25ms/step - loss: 1.7999 - accuracy: 0.5151
Epoch 2/10
57/57 [==============================] - 1s 12ms/step - loss: 0.8884 - accuracy: 0.7248
Epoch 3/10
57/57 [==============================] - 1s 12ms/step - loss: 0.6823 - accuracy: 0.7922
Epoch 4/10
57/57 [==============================] - 1s 12ms/step - loss: 0.5348 - accuracy: 0.8152
Epoch 5/10
57/57 [==============================] - 1s 12ms/step - loss: 0.3854 - accuracy: 0.8707
Epoch 6/10
57/57 [==============================] - 1s 12ms/step - loss: 0.3735 - accuracy: 0.8538
Epoch 7/10
57/57 [==============================] - 1s 12ms/step - loss: 0.3185 - accuracy: 0.8802
Epoch 8/10
57/57 [==============================] - 1s 12ms/step - loss: 0.2553 - accuracy: 0.9170
Epoch 9/10
57/57 [==============================] - 1s 12ms/step - loss: 0.2135 - accuracy: 0.9233
Epoch 10/10
57/57 [==============================] - 1s 12ms/step - loss: 0.2089 - accuracy: 0.9279


## Train model

In [ ]:
model.fit(x_train,y_train,x_val,y_val,batch_size=128, data_augmentation=False)
model_json = model.model.to_json()
f = open('model.json', 'w')
f.write(model_json)
f.close()
model.model.save_weights('model_weights.h5')

In [ ]:
model.model = tensorflow.keras.models.load_model('MLmodel.hdf5')

In [ ]:
report = model.classification_report(x_test,y_test)
print(report)

In [ ]:
report = model.classification_report(x_test,y_test)
print(report)

In [ ]:
np.argmax(y_test,1)
np.unique(np.argmax(y_test,1),return_counts=True)

In [ ]:
from matplotlib import pylab as plt
import itertools
M = model.confusion_matrix(x_test,y_test)
M = np.int32(M[:15,:15])
print(M)
print("----")
print(np.sum(M,1))
print("----")
M = (M.T/np.sum(M,1)).T

print(M)

f, ax = plt.subplots(1,1,figsize=(15,15))
ax.imshow(M,cmap='Purples')

# Major ticks
ax.set_xticks(np.arange(0, 15, 1));
ax.set_yticks(np.arange(0, 15, 1));

# Labels for major ticks
ax.set_xticklabels(np.arange(1, 15, 1));
ax.set_yticklabels(np.arange(1, 15, 1));

# Minor ticks
ax.set_xticks(np.arange(-.5, 14, 1), minor=True);
ax.set_yticks(np.arange(-.5, 14, 1), minor=True);

ax.grid(which='minor', color='black', linestyle='-', linewidth=2)


ax.set_xlabel('Predictions')
ax.set_ylabel('Truth')
upper_thresh = 0.5
format(M[i, j], '.2f')
for i,j in itertools.product(range(15),range(15)):
    ax.text(j, i, format(M[i, j], '.2f'), horizontalalignment="center", fontsize=20,
                 color="white" if (M[i, j] > upper_thresh ) else "black")

plt.show()

In [ ]:
from matplotlib import pylab as plt
import itertools
M = model.confusion_matrix(x_test,y_test)
M = np.int32(M[:15,:15])

M = (M.T/np.sum(M,1)).T


f, ax = plt.subplots(1,1,figsize=(15,15))
ax.imshow(M,cmap='Purples')

# Major ticks
ax.set_xticks(np.arange(0, 15, 1));
ax.set_yticks(np.arange(0, 15, 1));

# Labels for major ticks
ax.set_xticklabels(np.arange(1, 15, 1));
ax.set_yticklabels(np.arange(1, 15, 1));

# Minor ticks
ax.set_xticks(np.arange(-.5, 14, 1), minor=True);
ax.set_yticks(np.arange(-.5, 14, 1), minor=True);

ax.grid(which='minor', color='black', linestyle='-', linewidth=2)


ax.set_xlabel('Predictions')
ax.set_ylabel('Truth')
upper_thresh = 0.5
format(M[i, j], '.2f')
for i,j in itertools.product(range(15),range(15)):
    ax.text(j, i, format(M[i, j], '.2f'), horizontalalignment="center", fontsize=20,
                 color="white" if (M[i, j] > upper_thresh ) else "black")

plt.show()

In [ ]:
y_pred = model.model.predict_classes(x_test)
y_test_ = np.argmax(y_test,1)

In [ ]:
print(y_pred.shape)
print(y_test_.shape)

for pred_, true_ in zip(y_pred,y_test_):
    print(pred_, " - ", true_)